In [1]:
## Import basic and advanced modules
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# Extract top courses of intended providers
df = pd.read_csv('08_Jan_8_providers.csv')
# Only brentwood courses
df = df[df.provider=='Brentwood Open learning College'].reset_index(drop=True)
# Get the top 22 courses sorted by sales
df_top_22 = df.sort_values(by='sold',ascending=False).reset_index(drop=True).iloc[0:22,:]
"These are the top 22 courses for those we will count keywords frequency and density"
df_top_22.head()

,title,link,provider,subtitle,price,sold,id
0,Leadership & Management,https://www.reed.co.uk/courses/leadership-mana...,Brentwood Open learning College,Leadership and Management Level 5 endorsed by ...,10.0,3028,244242
1,Digital Marketing Diploma,https://www.reed.co.uk/courses/digital-marketi...,Brentwood Open learning College,Digital Marketing Advanced Level 5 Endorsed by...,10.0,1054,237269
2,Cyber Security,https://www.reed.co.uk/courses/cyber-security/...,Brentwood Open learning College,Cyber Security Diploma Level 5 Endorsed by ABC...,10.0,912,247335
3,Diet and Nutrition Advanced Diploma Level 5,https://www.reed.co.uk/courses/diet-and-nutrit...,Brentwood Open learning College,Diet and Nutrition Level 5 Endorsed by ABC Awa...,10.0,807,97418
4,Facilities Management Diploma,https://www.reed.co.uk/courses/facilities-mana...,Brentwood Open learning College,Facilities Management Diploma Level 6 endorsed...,10.0,549,247081


In [3]:
# Extract data from those 22 links
# Initialize variables to be extracted
overview = []
achievement = []
des = []
who_is_course = []
requirments = []
career_path = []

# Make request to those links to extract data
for lnk in df_top_22.link:
    request = requests.get(lnk)
    soup = BeautifulSoup(request.content, 'html.parser')
    

    # Overview
    over_tag = soup.findAll('div', class_="collapsable-container scroll-point")[0].findAll('div')[1]
    overview.append(over_tag.text if over_tag is not None else 'missing')

#     # Achievement
#     if soup.findAll('div', class_="collapsable-container scroll-point")[1].h2.text=='Achievement':
#         achievement.append(soup.findAll('div', class_="collapsable-container scroll-point")[1].findAll('div')[1].text.strip()) 
#     else:
#         achievement.append('missing')

    # Description
    des_tag = soup.findAll('div', class_="collapsable-container scroll-point")[-5].findAll('div')[1]
    des.append(des_tag.text.strip() if des_tag is not None else 'missing')
    
    # Who is this course for
    who_tag = soup.findAll('div', class_="collapsable-container scroll-point")[-4].findAll('div')[1]
    who_is_course.append(who_tag.text.strip() if who_tag is not None else 'missing')
    
    # Requirments
    req_tag = soup.findAll('div', class_="collapsable-container scroll-point")[-3].findAll('div')[1]
    requirments.append(req_tag.text.strip() if req_tag is not None else 'missing')

    # Career path
    career_path_tag = soup.findAll('div', class_="collapsable-container scroll-point")[-2].findAll('div')[1]
    career_path.append(career_path_tag.text.strip() if career_path_tag is not None else 'missing')
    
# Create a df off those scraped variables
df_top_22_extracted = pd.DataFrame({'overview':overview,'description':des,'who_is_this_course':who_is_course,
                    'requirments':requirments,'career_path':career_path})

In [4]:
# Let's look what we have
df_top_22_extracted.head()

,overview,description,who_is_this_course,requirments,career_path
0,Leadership & Management Diploma. Assessment a...,Leadership & Management Diploma Course Benefit...,Leadership & Management Diploma A high-level D...,Leadership & Management Diploma There is no pa...,Leadership & Management Diploma An advanced Le...
1,Digital Marketing Diploma Endorsed Diploma Le...,"Digital Marketing Diploma Worldwide, Digital M...",Digital Marketing Diploma While millions of pe...,Digital Marketing Diploma There is no particul...,Digital Marketing Diploma Upon successful comp...
2,Cyber Security Endorsed cyber security diplom...,Cyber Security Course Benefits Studying for an...,Cyber Security Our exclusive Level 5 Cyber Sec...,Cyber Security A strong working knowledge of ...,Cyber Security Studying for an advanced Level ...
3,Diet and Nutrition Advanced Diploma Level 5 D...,Diet and Nutrition Advanced Diploma Level 5 Co...,Diet and Nutrition Diploma Level 5 This advanc...,Diet and Nutrition Diploma Level 5 We work har...,Diet and Nutrition Diploma Level 5 Upon succes...
4,Facilities Management Diploma Study Faciliti...,Facilities Management Diploma Facilities Mana...,Facilities Management Diploma It’s becoming in...,Facilities Management Diploma We do not ask f...,Facilities Management Diploma Advanced facilit...


## **Now we have top 22 courses of Brentwood's courses. We would like to count the occurances of a keyword mentioned in the sections overview, description, who_is_this_course, requirments, career_path. And finally create a variable 'sum' indicating total occurances for those sections.**

In [5]:
# Read the keywords file
df_key = pd.read_excel('keywords_to_match.xlsx')
df_key

,keywords
0,Leadership
1,Digital marketing
2,Cyber security
3,Diet/Nutrition
4,Management/ facilities management
5,Business/ Business analysis
6,Business Management/Management
7,Construction
8,Accounting
9,HR/Management/HR management


## So these are the keywords to match for top 22 courses.

In [6]:
# Create a function that will count the frequency of keywords we like to count for each courses
def match(keyword,row):
    df = df_top_22_extracted.apply(lambda x: x.str.count(keyword,flags=re.I))
    # df2 = df1.loc[:,~df1.columns.str.contains('sold')].apply(lambda x: x.str.count(word,flags=re.I))
    # Count the total occurance
    df['sum'] = df.sum(axis=1)
    # Merge link, title and sold column
    df = pd.concat([df_top_22[['link','title','sold']],df],axis=1)
    # Create the keyword column we entered
    df.insert(loc=3,value=keyword,column='keyword')
    # Save as csv file
    #return df.iloc[[row]].to_csv(f'{name}.csv',index=False)
    return df.iloc[[row]]

In [7]:
# Calculateall the keywords frequency
df0 = match('leadership',0)
df1 = match('digital marketing',1)
df2 = match ('cyber security',2)

df3_1 = match ('diet',3)
df3_2 = match ('nutrition',3)

df4_1 = match ('management',4)
df4_2 = match ('facilities management',4)

df5_1 = match ('business',5)
df5_2 = match ('business analysis',5)


df6_1 = match ('management',6)
df6_2 = match ('business management',6)

df7 = match ('construction',7)
df8 = match ('accounting',8)

df9_1 = match ('hr',9)
df9_2 = match ('hr management',9)
df9_3 = match ('management',9)

df10_1 = match ('operations',10)
df10_2 = match ('operations management',10)

df11_1 = match ('travel',11)
df11_2 = match ('tourism',11)

df12 = match ('construction',12)
df13 = match ('bookkeeping',13)
df14 = match ('project management',14)
df15 = match ('supply chain',15)

df16_1 = match ('site management',16)
df16_2 = match ('construction',16)

df17 = match ('event management',17)
df18 = match ('project management',18)

df19_1 = match ('teaching',19)
df19_2 = match ('teaching assiatant',19)

df20 = match ('customer service',20)

df21_1 = match ('sales',21)
df21_2 = match ('sales management',21)

In [8]:
# Concat all the variables
df = pd.concat([df0,df1,df2,df3_1,df3_2,df4_1,df4_2,df5_1,df5_2,
               df6_1,df6_2,df7,df8,df9_1,df9_2,df9_3,df10_1,df10_2,df11_1,df11_2,
                df12,df13,df14,df15,df16_1,df16_2,df17,df18,df19_1,df19_2,df20,df21_1,df21_2],axis=0).reset_index(drop=True)
# So what we have got
df.head(10)

,link,title,sold,keyword,overview,description,who_is_this_course,requirments,career_path,sum
0,https://www.reed.co.uk/courses/leadership-mana...,Leadership & Management,3028,leadership,11,18,4,1,3,37
1,https://www.reed.co.uk/courses/digital-marketi...,Digital Marketing Diploma,1054,digital marketing,9,17,4,2,2,34
2,https://www.reed.co.uk/courses/cyber-security/...,Cyber Security,912,cyber security,13,18,6,1,3,41
3,https://www.reed.co.uk/courses/diet-and-nutrit...,Diet and Nutrition Advanced Diploma Level 5,807,diet,14,24,4,1,3,46
4,https://www.reed.co.uk/courses/diet-and-nutrit...,Diet and Nutrition Advanced Diploma Level 5,807,nutrition,13,25,3,1,11,53
5,https://www.reed.co.uk/courses/facilities-mana...,Facilities Management Diploma,549,management,19,35,4,1,2,61
6,https://www.reed.co.uk/courses/facilities-mana...,Facilities Management Diploma,549,facilities management,13,10,4,1,2,30
7,https://www.reed.co.uk/courses/business-analys...,Business Analysis,516,business,14,40,7,1,7,69
8,https://www.reed.co.uk/courses/business-analys...,Business Analysis,516,business analysis,6,12,4,1,3,26
9,https://www.reed.co.uk/courses/business-manage...,Business Management - Course,511,management,12,23,2,1,3,41
